
# Local RAG Demo (Jupyter) — Ollama Embeddings + FAISS


Try these in a terminal:
```bash
ollama list
ollama pull nomic-embed-text
ollama pull gemma:2b
```


In [12]:

# If needed, install dependencies. (Safe to re-run.)
%pip -q install  faiss-cpu numpy pandas tqdm requests



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:

# === Configuration (change these as needed) ===
EMBED_MODEL = "embeddinggemma:latest"   
#EMBED_MODEL = "nomic-embed-text"

WORDS_PER_CHUNK = 300
OVERLAP_WORDS   = 60
TOPK            = 5

# Toggle downloads off if you want to stay strictly offline (then we'll use tiny built-in samples).
DOWNLOAD_FROM_WEB = True

# A small selection of long classics (public domain). Even 2–3 will give you 300+ chunks.

GUTENBERG_BOOKS = {
    "Moby-Dick": "https://www.gutenberg.org/files/2701/2701-0.txt",
    "Pride and Prejudice": "https://www.gutenberg.org/files/1342/1342-0.txt",
    "Frankenstein": "https://www.gutenberg.org/files/84/84-0.txt",
    "Alice in Wonderland": "https://www.gutenberg.org/cache/epub/11/pg11.txt",
    "Dracula": "https://www.gutenberg.org/files/345/345-0.txt",
    "A Tale of Two Cities": "https://www.gutenberg.org/files/98/98-0.txt",
    "The Great Gatsby": "https://www.gutenberg.org/cache/epub/64317/pg64317.txt",
    "Adventures of Sherlock Holmes": "https://www.gutenberg.org/files/1661/1661-0.txt",
    "War and Peace": "https://www.gutenberg.org/files/2600/2600-0.txt",
    "Jane Eyre": "https://www.gutenberg.org/files/1260/1260-0.txt",
    "The Picture of Dorian Gray": "https://www.gutenberg.org/files/174/174-0.txt",
    "Crime and Punishment": "https://www.gutenberg.org/files/2554/2554-0.txt",
    "Wuthering Heights": "https://www.gutenberg.org/files/768/768-0.txt",

}


GUTENBERG = [
    (title, url) for title, url in GUTENBERG_BOOKS.items()
]

CORPUS_DIR = "corpus_jupyter"


In [14]:

import os, re, json, textwrap
from pathlib import Path
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm

import ollama
import faiss

import truststore
truststore.inject_into_ssl()

# Minimal helper: strip Project Gutenberg boilerplate if found.
START_MARK = re.compile(r"\*\*\* START OF (THIS|THE) PROJECT GUTENBERG EBOOK .* \*\*\*", re.I)
END_MARK   = re.compile(r"\*\*\* END OF (THIS|THE) PROJECT GUTENBERG EBOOK .* \*\*\*", re.I)

def strip_gutenberg_boilerplate(txt: str) -> str:
    start = START_MARK.search(txt)
    end = END_MARK.search(txt)
    if start and end and end.start() > start.end():
        return txt[start.end():end.start()].strip()
    # Fallback heuristics
    txt = re.sub(r"(?s)^.*?Project Gutenberg.*?eBook.*?\n", "", txt, flags=re.I)
    txt = re.sub(r"(?s)End of Project Gutenberg.*$", "", txt, flags=re.I)
    return txt.strip()

def l2_normalize(mat: np.ndarray) -> np.ndarray:
    norms = np.linalg.norm(mat, axis=1, keepdims=True) + 1e-12
    return mat / norms


In [15]:
# === Check if pre-built artifacts exist; if so, load and skip heavy work ===
import json as _json

ARTIFACTS_DIR = "rag_artifacts"
_artifact_files = ["chunks.json", "embeddings.npy", "faiss_index.bin", "config.json"]
ARTIFACTS_LOADED = all((Path(ARTIFACTS_DIR) / f).exists() for f in _artifact_files)

if ARTIFACTS_LOADED:
    print("✅ Found pre-built artifacts! Loading from disk …")
    with open(Path(ARTIFACTS_DIR) / "chunks.json", encoding="utf-8") as _f:
        chunks = _json.load(_f)
    emb = np.load(str(Path(ARTIFACTS_DIR) / "embeddings.npy"))
    index = faiss.read_index(str(Path(ARTIFACTS_DIR) / "faiss_index.bin"))
    print(f"   {len(chunks)} chunks | embeddings {emb.shape} | FAISS {index.ntotal} vectors")
    print("   Skipping download, chunking, embedding, and index-build cells.")
else:
    print("⏳ Artifacts not found — will build from scratch.")

✅ Found pre-built artifacts! Loading from disk …
   8509 chunks | embeddings (8509, 768) | FAISS 8509 vectors
   Skipping download, chunking, embedding, and index-build cells.


In [16]:
if not ARTIFACTS_LOADED:
    Path(CORPUS_DIR).mkdir(parents=True, exist_ok=True)

    docs = []  # list of dicts: {title, text, path}

    if DOWNLOAD_FROM_WEB:
        for title, url in GUTENBERG:
            out_path = Path(CORPUS_DIR) / f"{title.replace(' ', '_')}.txt"
            if not out_path.exists():
                print(f"Downloading: {title}")
                try:
                    r = requests.get(url, timeout=60)
                    r.raise_for_status()
                    clean = strip_gutenberg_boilerplate(r.text)
                    out_path.write_text(clean, encoding="utf-8")
                except Exception as e:
                    print(f"  Failed ({e}); skipping.")
            else:
                print(f"Exists: {out_path.name}")
            if out_path.exists():
                docs.append({
                    "title": title,
                    "text": out_path.read_text(encoding="utf-8", errors="ignore"),
                    "path": str(out_path)
                })

    print(f"Loaded {len(docs)} docs")
else:
    print("⏩ Skipped (artifacts already loaded)")

⏩ Skipped (artifacts already loaded)


In [17]:
if not ARTIFACTS_LOADED:
    # print no of words in each document
    for d in docs:
        num_words = len(d["text"].split())
        print(f"{d['title']}: {num_words} words")
else:
    print("⏩ Skipped (artifacts already loaded)")

⏩ Skipped (artifacts already loaded)


In [18]:
if not ARTIFACTS_LOADED:
    # --- Chunking ---
    chunks = []  # list of dicts: {id, title, text, preview, source_path, chunk_index}
    for d in docs:
        words = d["text"].split()
        if not words:
            continue
        step = max(1, WORDS_PER_CHUNK - OVERLAP_WORDS)
        idx = 0
        chunk_i = 0
        while idx < len(words):
            segment = words[idx:idx+WORDS_PER_CHUNK]
            if len(segment) < max(60, WORDS_PER_CHUNK//4):
                break
            text_seg = " ".join(segment)
            chunks.append({
                "id": f"{Path(d['title']).name.replace(' ', '_')}#chunk{chunk_i}",
                "title": d["title"],
                "text": text_seg,
                "preview": text_seg[:400],
                "source_path": d["path"],
                "chunk_index": chunk_i,
            })
            chunk_i += 1
            idx += step

    print(f"Total chunks: {len(chunks)}")
    pd.DataFrame([
        {"id": c["id"], "title": c["title"], "preview": c["preview"][:140] + ("…" if len(c["preview"])>140 else "")}
        for c in chunks[:8]
    ])
else:
    print(f"⏩ Skipped — {len(chunks)} chunks already loaded from artifacts")

⏩ Skipped — 8509 chunks already loaded from artifacts


In [19]:

## Embed each chunk with your Ollama embedding model (one-by-one API).
## NOTE: Ensure EMBED_MODEL is a valid embeddings model in `ollama list`.
#emb_vectors = []
#for c in tqdm(chunks, desc=f"Embedding with {EMBED_MODEL}"):
#    resp = ollama.embeddings(model=EMBED_MODEL, prompt=c["text"])
#    v = np.asarray(resp["embedding"], dtype="float32")
#    emb_vectors.append(v)

In [20]:
if not ARTIFACTS_LOADED:
    from concurrent.futures import ThreadPoolExecutor, as_completed
    import time
    import numpy as np
    from tqdm import tqdm
    import ollama
    import faiss

    # Tune this based on your machine; 4–8 is usually the sweet spot.
    MAX_WORKERS = 6
    RETRIES = 3
    BACKOFF = 0.6  # seconds, linear backoff

    def embed_text(text: str) -> np.ndarray:
        last_err = None
        for attempt in range(1, RETRIES + 1):
            try:
                resp = ollama.Client(host="http://localhost:11434", trust_env=False).embeddings(model=EMBED_MODEL, prompt=text)
                return np.asarray(resp["embedding"], dtype="float32")
            except Exception as e:
                last_err = e
                if attempt < RETRIES:
                    time.sleep(BACKOFF * attempt)
        raise last_err

    # Parallelize over chunks while preserving original order
    emb_vectors = [None] * len(chunks)
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
        futures = {ex.submit(embed_text, c["text"]): i for i, c in enumerate(chunks)}
        for fut in tqdm(as_completed(futures), total=len(futures), desc=f"Embedding ({EMBED_MODEL})"):
            i = futures[fut]
            emb_vectors[i] = fut.result()
else:
    print(f"⏩ Skipped — embeddings already loaded from artifacts")

⏩ Skipped — embeddings already loaded from artifacts


In [21]:
if not ARTIFACTS_LOADED:
    emb = np.vstack(emb_vectors)  # shape (N, d)
    d = emb.shape[1]
    print("Embeddings shape:", emb.shape)

    # L2-normalize and build FAISS IP index (IP on unit-norm == cosine similarity)
    emb = l2_normalize(emb)
    index = faiss.IndexFlatIP(d)
    index.add(emb)
    print("FAISS index size:", index.ntotal)
else:
    print(f"⏩ Skipped — FAISS index already loaded ({index.ntotal} vectors)")

⏩ Skipped — FAISS index already loaded (8509 vectors)


In [22]:
if not ARTIFACTS_LOADED:
    # === Save all time-consuming artifacts to disk for fast Flask deployment ===
    import json, pickle

    ARTIFACTS_DIR = "rag_artifacts"
    Path(ARTIFACTS_DIR).mkdir(parents=True, exist_ok=True)

    # 1. Save chunks metadata as JSON
    with open(Path(ARTIFACTS_DIR) / "chunks.json", "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False)
    print(f"Saved {len(chunks)} chunks to {ARTIFACTS_DIR}/chunks.json")

    # 2. Save normalized embeddings as numpy array
    np.save(Path(ARTIFACTS_DIR) / "embeddings.npy", emb)
    print(f"Saved embeddings shape {emb.shape} to {ARTIFACTS_DIR}/embeddings.npy")

    # 3. Save FAISS index
    faiss.write_index(index, str(Path(ARTIFACTS_DIR) / "faiss_index.bin"))
    print(f"Saved FAISS index ({index.ntotal} vectors) to {ARTIFACTS_DIR}/faiss_index.bin")

    # 4. Save config so Flask app knows model name, chunk params, etc.
    config = {
        "EMBED_MODEL": EMBED_MODEL,
        "WORDS_PER_CHUNK": WORDS_PER_CHUNK,
        "OVERLAP_WORDS": OVERLAP_WORDS,
        "TOPK": TOPK,
    }
    with open(Path(ARTIFACTS_DIR) / "config.json", "w") as f:
        json.dump(config, f, indent=2)
    print(f"Saved config to {ARTIFACTS_DIR}/config.json")

    print("\n✅ All artifacts saved!")
else:
    print("⏩ Skipped — artifacts already exist on disk")

⏩ Skipped — artifacts already exist on disk


In [23]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

## Step-by-Step Breakdown: `expand_with_neighbors_simple()`

This function takes the top few chunks found by FAISS and expands them by grabbing neighboring chunks, then merges contiguous ones.

### Input Parameters:
- **`I`**: Indices array of shape `(1, num_hits)` — contains global chunk indices of matched chunks in order of relevance
- **`D`**: Distances array of shape `(1, num_hits)` — contains similarity scores for each match
- **`chunks`**: List of all chunk dictionaries with metadata
- **`neighbors`**: How many chunks before/after each hit to grab (e.g., 1 = grab ±1, so [-1, 0, +1])
- **`max_out`**: Maximum total chunks to return (safety limit)

### Step 1: Initialize Tracking Structures
```python
seen = set()            # Track which chunks we've already included (avoid duplicates)
ordered_idxs = []       # Maintain order of chunks as we add them
```

### Step 2: Walk Through Hits in Relevance Order
For each hit from FAISS (best match first):
1. Extract the chunk metadata and its document path + chunk index
2. For each neighbor offset (-neighbors to +neighbors):
   - Try to find that neighbor in the `KEY_TO_IDX` lookup map
   - If it exists AND we haven't seen it: add it to our list
   - If we've hit max chunks: stop immediately

**Example:** If hit is chunk 5 and neighbors=1:
- Check chunks 4, 5, 6
- If they exist and we haven't grabbed them, add all three

### Step 3: Sort by Document and Chunk Order
```python
ordered_idxs.sort(key=lambda j: (chunks[j]["source_path"], chunks[j]["chunk_index"]))
```
Why? Because we added chunks in relevance order, but now we need them in document order to detect contiguous ranges.

**Example Result:** 
- Before: [5, 4, 12, 6]  (relevance order from FAISS)
- After: [4, 5, 6, 12]  (document/chunk order)

### Step 4: Merge Contiguous Chunks into Spans
Walk through sorted chunks and group consecutive ones:

1. **Start a span** with the first chunk
2. **For each next chunk:**
   - If it's from the **same document** AND its chunk_index is **exactly one more** than the current span's end:
     - **Extend** the existing span (increment end_ci, add chunk to list)
   - Otherwise:
     - **Save** the previous span
     - **Start a new span** with this chunk
3. **Don't forget** the last span

**Example:**
- Input chunks (sorted): 4, 5, 6 (same doc, chunk_index 4→5→6)
- Output span: `{start_ci: 4, end_ci: 6, idxs: [chunk4, chunk5, chunk6]}`

### Step 5: Build Final Context Objects
For each merged span:
1. **Join texts** of all chunks in the span with "\n\n"
2. **Create metadata**: title, source_path, chunk range, word count, span size
3. **Return** list of context objects ready for the LLM

**Output Example:**
```python
{
    "title": "Pride and Prejudice",
    "source_path": "/path/to/file",
    "start_chunk": 4,
    "end_chunk": 6,
    "text": "[full merged text]",
    "approx_words": 900,  # sum of all chunks
    "span_count": 3
}
```


## Step-by-Step Breakdown: `search_windowed()`

This is the main search orchestrator. It takes a user query and returns both raw FAISS hits and intelligently expanded contexts.

### Input Parameters:
- **`query`**: User's natural language question/search string
- **`topk`**: Number of initial FAISS hits to retrieve (default: TOPK=5)
- **`max_out`**: Maximum chunks in final output (default: 8)

### Step 1: Embed the Query
```python
q_vec = ollama.Client().embeddings(model=EMBED_MODEL, prompt=query)
q_vec = q_vec / (np.linalg.norm(q_vec) + 1e-12)  # L2 normalize
```

**What's happening:**
- Convert the query text into a numerical vector using the Ollama embedding model
- Normalize it so its length = 1 (this makes cosine similarity = dot product in FAISS)
- The `+1e-12` prevents division by zero for edge cases

**Output:** A 384-dimensional vector representing the query's meaning

### Step 2: Search FAISS for Top-K Matches
```python
D, I = index.search(q_vec.reshape(1, -1), topk)
```

**What happens:**
- FAISS searches for the `topk` most similar chunks to your query
- Returns:
  - **`I[0]`**: Indices of top-k chunks (e.g., [5, 12, 3, 8, 1])
  - **`D[0]`**: Similarity scores (e.g., [0.89, 0.76, 0.65, 0.58, 0.52])

**Example interpretation:**
- Chunk #5 is most similar (score 0.89)
- Chunk #12 is second (score 0.76)
- etc., ordered by relevance

### Step 3: Decide Neighbor Window Size (Heuristic)
```python
neighbors = 1 if WORDS_PER_CHUNK >= 260 else 2
```

**Logic:**
- **Large chunks (≥260 words):** Each chunk already has lots of context, so grab only ±1 neighbors
- **Small chunks (<260 words):** Need more context, so grab ±2 neighbors

**Why?** Prevents context gaps while keeping output reasonable.

### Step 4: Decide How Many Hits to Expand (Heuristic)
```python
top1 = float(D[0][0])           # Best match score
top2 = float(D[0][1]) if len(D[0]) > 1 else 0.0
margin = top1 - top2            # Gap between 1st and 2nd

init_topk = 1 if (top1 >= 0.35 and margin >= 0.05) else min(3, topk)
```

**Logic:**
- If **top match is confident** (score ≥ 0.35) AND **clearly wins** (margin ≥ 0.05):
  - Use only `1` hit (the best one is enough)
- Otherwise:
  - Use up to `3` hits (cast a wider net)

**Intuition:**
- Query: "Who is Sherlock Holmes?" → top hit: Sherlock bio (0.89) vs. second (0.70) = margin 0.19 → Use 1 hit
- Query: "Victorian clothing" → top hit: costume desc (0.75) vs. second (0.72) = margin 0.03 → Use 3 hits (competing sources)

### Step 5: Slice to Get Only Selected Hits
```python
D_init = np.array([D[0][:init_topk]])
I_init = np.array([I[0][:init_topk]])
```

**Example:**
- If `init_topk=1`: Take only the top 1 hit's index and score
- If `init_topk=3`: Take the top 3 hits

**Result:** Focused arrays for neighbor expansion

### Step 6: Expand with Neighbors and Merge
```python
contexts = expand_with_neighbors_simple(I_init, D_init, chunks, 
                                        neighbors=neighbors, max_out=max_out)
```

**What happens:**
- Call the function we explained above
- Returns: List of merged context objects ready for the LLM

**Example output:**
```python
[
    {
        "title": "Adventures of Sherlock Holmes",
        "source_path": "...",
        "start_chunk": 5,
        "end_chunk": 7,
        "text": "[1200 words of merged chunks 5, 6, 7]",
        "approx_words": 1200,
        "span_count": 3
    },
    {
        "title": "A Scandal in Bohemia",  # Different doc, gap in indexes
        "source_path": "...",
        "start_chunk": 120,
        "end_chunk": 120,
        "text": "[350 words]",
        "approx_words": 350,
        "span_count": 1
    }
]
```

### Step 7: Build Display DataFrame (All K Hits)
```python
rows = []
for score, idx_ in zip(D[0].tolist(), I[0].tolist()):
    m = chunks[idx_]
    rows.append({
        "score": round(score, 3),
        "id": m["id"],
        "title": m["title"],
        "source_path": m["source_path"],
        "chunk_index": m["chunk_index"],
        "preview": (m["preview"][:220] + "…") if len(m["preview"])>220 else m["preview"],
    })
df_hits = pd.DataFrame(rows)
```

**Purpose:** Show all `topk` FAISS hits for transparency/debugging

**Example output table:**

| score | id              | title          | chunk_index | preview         |
|-------|-----------------|----------------|-------------|-----------------|
| 0.891 | Holmes#chunk4   | Sherlock...    | 4           | "The adventure began when..."  |
| 0.756 | Holmes#chunk12  | Sherlock...    | 12          | "Watson exclaimed in..."        |
| 0.654 | Holmes#chunk18  | Sherlock...    | 18          | "The criminal mastermind..."   |

### Step 8: Return Results
```python
return df_hits, contexts
```

**Returns:**
1. **`df_hits`**: DataFrame showing all raw FAISS hits (for debugging)
2. **`contexts`**: List of merged context objects (for LLM input)

---

## Complete Example Flow

**Query:** `"Elizabeth's love for Darcy"`

1. ✅ Embed query → vector [0.341, -0.129, ..., 0.554]
2. ✅ FAISS search → I=[42, 108, 33, 5, 71], D=[0.87, 0.65, 0.58, 0.52, 0.49]
3. ✅ Decision: score=0.87 ≥ 0.35? YES. margin=(0.87-0.65)=0.22 ≥ 0.05? YES. → Use 1 hit
4. ✅ Expand chunk 42 with neighbors (±1) → get chunks [41, 42, 43]
5. ✅ Merge: All same doc, chunks 41→42→43 are contiguous → span {start:41, end:43}
6. ✅ Return: df_hits with all 5 raw hits, contexts with 1 merged span (3 chunks joined)

**Output to LLM:** ~900 words of continuous context about Elizabeth & Darcy's relationship


## Visual System Flow

### High-Level Architecture Diagram

```
┌─────────────────────────────────────────────────────────────────────┐
│                     USER QUERY                                      │
│              "Elizabeth's love for Darcy"                          │
└────────────────────────────┬────────────────────────────────────────┘
                             │
                             ▼
                  ┌──────────────────────┐
                  │  Embed Query         │
                  │  (Ollama model)      │
                  │  384-dim vector      │
                  └──────────────────────┘
                             │
                             ▼
                  ┌──────────────────────────────────────┐
                  │  FAISS ANN Search (topk=5)           │
                  │                                      │
                  │  Returns:                            │
                  │  I = [42, 108, 33, 5, 71]  (indices)│
                  │  D = [0.87, 0.65, 0.58...] (scores) │
                  └──────────────────────────────────────┘
                             │
                  ┌──────────┴──────────┐
                  │                     │
        Save (for debugging)   Decide Expansion Strategy
                  │                     │
                  │                ┌────▼─────────┐
                  │                │ Is top match │
                  │                │  strong &    │
                  │                │  confident?  │
                  │                └────┬─────────┘
                  │                     │
                  │            ┌────────┴────────┐
                  │            │                 │
                  │         YES│                 │NO
                  │            │                 │
                  │       1 hit │          1-3 hits
                  │            │                 │
                  │            └────────┬────────┘
                  │                     │
                  │            ┌────────▼──────────┐
                  │            │  Expand Neighbors │
                  │            │  (±1 or ±2 chunks)│
                  │            └─────────┬────────┘
                  │                      │
                  │            ┌─────────▼────────┐
                  │            │ Merge Contiguous │
                  │            │ Chunks into Spans│
                  │            └────────┬────────┘
                  │                     │
        ┌─────────▼──────────┐  ┌──────▼──────────────┐
        │   df_hits table    │  │  Expanded Contexts  │
        │ (all 5 raw hits)   │  │  (merged spans)     │
        └────────────────────┘  └──────┬──────────────┘
                                       │
                        ┌──────────────▼──────────────┐
                        │  Ready for LLM / RAG System │
                        │  (Better context window)    │
                        └─────────────────────────────┘
```

---

## Key Design Decisions Explained

### 1. Why Use Two Different Functions?

**`expand_with_neighbors_simple()`** = Context Expansion
- Knows how to walk FAISS results and grab neighbors
- Knows how to merge contiguous chunks
- Pure chunk manipulation

**`search_windowed()`** = Intelligence Layer
- Decides *how many* hits to expand (smart heuristics)
- Decides *how wide* the neighbor window should be
- Coordinates the overall search strategy

**Benefit:** Separation of concerns. Easy to swap in different expansion strategies.

---

### 2. Why Search FAISS for Topk=5 but Only Expand 1-3?

**Problem:** More hits = more API calls, higher latency, more context to process

**Solution:** Confidence heuristics
- If the top hit is **very confident**, use only that one
- If results are **ambiguous** (close scores), use multiple hits for coverage

**Example:**
- Query: "Who wrote Moby Dick?" → Search shows ~90% certainty = 1 hit enough
- Query: "Whaling techniques" → Multiple chapters discuss it (~similar scores) = 3 hits better

---

### 3. Why Merge Contiguous Chunks?

**Problem:** If you expand hits, you get overlapping chunks

**Example:**
- FAISS hit: chunk 5
- Expand ±1: chunks 4, 5, 6
- If next hit is chunk 6, you'd have: [4, 5, 6] + [5, 6, 7] = duplication!

**Solution:** After gathering all neighbors, merge contiguous ones
- Input: [4, 5, 6, 7] → Output: single span 4-7 with merged text
- No duplication, no gaps, clean output

---

### 4. Why KEY_TO_IDX Lookup Map?

```python
KEY_TO_IDX = {(source_path, chunk_index): global_idx ...}
```

**Problem:** Given a chunk's (document, chunk_number), how do you find its global index?

**Before:** Scan through all 300+ chunks every time
**After:** O(1) dictionary lookup

**Critical for performance** when expanding thousands of queries.

---

## Decision Tree: Smart Expansion

```
Query comes in
    │
    ├─ Best match score = 0.89, Second = 0.70 (margin=0.19)
    │  → Strong & confident
    │  → expand_topk = 1
    │  → Reasoning: We're very sure of the answer
    │
    ├─ Best match score = 0.75, Second = 0.71 (margin=0.04)
    │  → Weak confidence signal
    │  → expand_topk = 3
    │  → Reasoning: Ambiguous, need alternative perspectives
    │
    └─ Chunk size very large (400+ words)
       → neighbors = 1 (less context needed)
       OR
       Chunk size small (150 words)
       → neighbors = 2 (more context helpful)
```

---

## Common Failure Modes & How This Avoids Them

| Problem | How This Design Helps |
|---------|----------------------|
| **Too much context** → LLM confusion | Smart heuristic limits to 1-3 base hits |
| **Chunk boundaries cut sentences** | Neighbor expansion gives full context |
| **Duplicate chunks in output** | Merging deduplicates seamlessly |
| **Slow retrieval** | topk=5 before filtering (not 100) |
| **Missing context** | Neighbors handle cross-chunk dependencies |
| **Document gaps** | Multiple sources if confidence is low |

---

## Example Walkthrough: Complete Query Flow

```
INPUT: query="What is the white whale in Moby Dick?"

STEP 1: Embed
  → Vector: [0.23, -0.41, ..., 0.67] (384 dims)

STEP 2: FAISS Search topk=5
  → I = [17, 45, 12, 88, 3]
  → D = [0.91, 0.58, 0.52, 0.49, 0.44]

STEP 3: Confidence Check
  → top1 = 0.91 ≥ 0.35? YES
  → margin = 0.91 - 0.58 = 0.33 ≥ 0.05? YES
  → init_topk = 1 (confident)

STEP 4: Window Size
  → WORDS_PER_CHUNK = 300 ≥ 260? YES
  → neighbors = 1

STEP 5: Expand Hit #17
  → Neighbors: chunk 16, 17, 18 (all exist, same doc)
  → All added (ordered_idxs = [16, 17, 18])
  → Hit max_out? NO, only 3 chunks so far

STEP 6: Merge Contiguous
  → ordered_idxs sorted: [16, 17, 18]
  → All contiguous (16→17→18)
  → Single span: {start_ci: 16, end_ci: 18, text: [merged]}

OUTPUT:
  df_hits: Table of all 5 raw matches (for transparency)
  contexts: [
    {
      title: "Moby-Dick",
      start_chunk: 16,
      end_chunk: 18,
      text: "...Chapter 41: The White Whale...(merged text of 900 words)...",
      approx_words: 900,
      span_count: 3
    }
  ]

→ LLM receives one cohesive 900-word context block
```


In [24]:
# --- Simplest windowed retrieval: no decay, preserve base-hit order ---


# Lookup map for (doc, chunk_index) -> global idx
KEY_TO_IDX = {(c["source_path"], c["chunk_index"]): gi for gi, c in enumerate(chunks)}


def expand_with_neighbors_simple(I, D, chunks, neighbors=1, max_out=8):
    """
    For the initial ANN hits (I, D), walk hits in rank order and
    append each hit plus its ±neighbors (within the same document).
    No scoring/decay; just preserve base-hit order and dedupe.
    Then merge contiguous chunks into spans and return merged contexts.
    """
    seen = set()
    ordered_idxs = []

    # Walk initial hits in order
    for gi in I[0]:
        c = chunks[int(gi)]
        doc = c["source_path"]
        ci  = c["chunk_index"]
        # hit and its neighbors
        for delta in range(-neighbors, neighbors + 1):
            key = (doc, ci + delta)
            j = KEY_TO_IDX.get(key)
            if j is None:
                continue
            if j not in seen:
                ordered_idxs.append(j)
                seen.add(j)
            if len(ordered_idxs) >= max_out:
                break
        if len(ordered_idxs) >= max_out:
            break

    # Sort by (doc, chunk_index) for merging
    ordered_idxs.sort(key=lambda j: (chunks[j]["source_path"], chunks[j]["chunk_index"]))

    # Merge contiguous chunks from the same doc into spans
    merged = []
    span = None
    for gi in ordered_idxs:
        c = chunks[gi]
        if (span 
            and c["source_path"] == span["source_path"] 
            and c["chunk_index"] == span["end_ci"] + 1):
            span["end_ci"] += 1
            span["idxs"].append(gi)
        else:
            if span: merged.append(span)
            span = {
                "title": c["title"],
                "source_path": c["source_path"],
                "start_ci": c["chunk_index"],
                "end_ci": c["chunk_index"],
                "idxs": [gi],
            }
    if span: merged.append(span)

    # Final contexts
    contexts = []
    for s in merged:
        text = "\n\n".join(chunks[i]["text"] for i in s["idxs"])
        contexts.append({
            "title": s["title"],
            "source_path": s["source_path"],
            "start_chunk": s["start_ci"],
            "end_chunk": s["end_ci"],
            "text": text,
            "approx_words": sum(len(chunks[i]["text"].split()) for i in s["idxs"]),
            "span_count": len(s["idxs"]),
        })
    return contexts


def search_windowed(query: str, topk: int = TOPK, max_out: int = 8):
    """
    1) ANN search (topk)
    2) Choose neighbor width (based on chunk size)
    3) Expand with simple ±neighbors (no decay) and merge
    Returns: (df_hits, contexts)
    """
    q_vec = np.asarray(ollama.Client(host="http://localhost:11434", trust_env=False).embeddings(model=EMBED_MODEL, prompt=query)["embedding"], dtype="float32")
    q_vec = q_vec / (np.linalg.norm(q_vec) + 1e-12)
    D, I = index.search(q_vec.reshape(1, -1), topk)

    # Heuristic: with ~300-word chunks, ±1 is usually enough; go ±2 if you shrink chunks later.
    neighbors = 1 if WORDS_PER_CHUNK >= 260 else 2

    # Margin heuristic to decide how many *base* hits to expand
    top1 = float(D[0][0])
    top2 = float(D[0][1]) if len(D[0]) > 1 else 0.0
    margin = top1 - top2
    init_topk = 1 if (top1 >= 0.35 and margin >= 0.05) else min(3, topk)

    D_init = np.array([D[0][:init_topk]])
    I_init = np.array([I[0][:init_topk]])

    contexts = expand_with_neighbors_simple(I_init, D_init, chunks, neighbors=neighbors, max_out=max_out)

    # Visible table of ANN hits (unchanged)
    rows = []
    for score, idx_ in zip(D[0].tolist(), I[0].tolist()):
        m = chunks[idx_]
        rows.append({
            "score": round(score, 3),
            "id": m["id"],
            "title": m["title"],
            "source_path": m["source_path"],
            "chunk_index": m["chunk_index"],
            "preview": (m["preview"][:220] + "…") if len(m["preview"])>220 else m["preview"],
        })
    df_hits = pd.DataFrame(rows)
    return df_hits, contexts


In [25]:
# %%
# --- Quick test query (non-interactive) ---
QUERY = "Elizabeth Bennet's changing feelings for Mr. Darcy"
df_hits, contexts = search_windowed(QUERY, topk=TOPK, max_out=8)

print("=== Initial ANN Hits ===")
display(df_hits)

print("\n=== Windowed Contexts (merged) ===")
display(pd.DataFrame([{
    "title": c["title"],
    "source": Path(c["source_path"]).name,
    "span": f"{c['start_chunk']}–{c['end_chunk']}",
    "approx_words": c["approx_words"],
    "span_count": c["span_count"],
    "preview": (c["text"][:220] + "…") if len(c["text"]) > 220 else c["text"]
} for c in contexts]))

=== Initial ANN Hits ===


,score,id,title,source_path,chunk_index,preview
0,0.517,Pride_and_Prejudice#chunk457,Pride and Prejudice,corpus_jupyter/Pride_and_Prejudice.txt,457,"Mr. Darcy!--and so it does, I vow. Well, any f..."
1,0.509,Pride_and_Prejudice#chunk83,Pride and Prejudice,corpus_jupyter/Pride_and_Prejudice.txt,83,"employed, Elizabeth could not help observing, ..."
2,0.490,Pride_and_Prejudice#chunk518,Pride and Prejudice,corpus_jupyter/Pride_and_Prejudice.txt,518,good as a lord! And a special licence--you mus...
3,0.486,Pride_and_Prejudice#chunk349,Pride and Prejudice,corpus_jupyter/Pride_and_Prejudice.txt,349,they are! He takes them now for people of fash...
4,0.480,Pride_and_Prejudice#chunk357,Pride and Prejudice,corpus_jupyter/Pride_and_Prejudice.txt,357,who had expected to find in her as acute and u...



=== Windowed Contexts (merged) ===


,title,source,span,approx_words,span_count,preview
0,Pride and Prejudice,Pride_and_Prejudice.txt,82–84,900,3,"Darcy were not such a great tall fellow, in co..."
1,Pride and Prejudice,Pride_and_Prejudice.txt,456–458,900,3,"but she does not know, no one can know, how mu..."
2,Pride and Prejudice,Pride_and_Prejudice.txt,517–518,600,2,"utter a syllable. Nor was it under many, many ..."
